In [1]:
# import
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0, "/media/noahhhhhh/dataScience/proj/competition/Mercedes_Benz_Greener_Manufacturing/python/utils/")
import pickle

import numpy as np
import pandas as pd
import gc

# utils
from clean import *
from encode import *
from featureEngineer import *
from model import *
from dups import *


########################### load ###########################
dt_all = pd.read_csv("/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/data/dt_all.csv")
dt_train_raw = pd.read_csv("/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/raw/train.csv")
dt_test_raw = pd.read_csv("/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/raw/test.csv")

# ids
ids_train = dt_train_raw.ID.values
ids_test = dt_test_raw.ID.values

# cols
cols_cat = dt_all.drop("ID", axis = 1).select_dtypes(include = ["object"]).columns.values.tolist()
cols_bin = dt_all.filter(regex = "^X").columns.values.tolist()
cols_bin = [x for x in cols_bin if x not in cols_cat]

# ## X. Model

dt_all_train = dt_all[dt_all.ID.isin(ids_train)]
dt_all_test = dt_all[dt_all.ID.isin(ids_test)]

print(dt_all_train.shape, dt_all_test.shape)


# # ### X.1. xgb
# with outlier
#params_cols_dict_best_xgb_withOutlier = featureSelectRun("xgb", dt_all, ids_train, ids_test, cols_cat, cols_bin, rounds = 200)
#pickle.dump(params_cols_dict_best_xgb_withOutlier, open( "/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/data/params_cols_dict_best_xgb_withOutlier.pkl", "wb"))

# with outlier and avg duplicates
dt_withOutlier_aggDupsMedian = dups("min", dt_all, cols_cat, cols_bin)

Using TensorFlow backend.


(4209, 665) (4209, 665)


In [9]:
dt_all_train = dt_all[dt_all["y"].notnull()]
dt_all_test = dt_all[~dt_all["y"].notnull()]

In [6]:
# folds
k = 10
bin_y = pd.qcut(dt_all_train.y.values, k, labels = [i for i in range(1, k + 1)]).astype("int64")
dt_all_train["Fold"] = bin_y

In [13]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = k, shuffle = True, random_state = 888)
dt_id_folds = pd.DataFrame()
for i, (ind_train, ind_valid) in enumerate(skf.split(dt_all_train, bin_y)):
    dt_id_folds = pd.concat([dt_id_folds
                            , pd.DataFrame({"ID": dt_all_train.iloc[ind_valid].ID.values
                                           , "Fold": i + 1})])

In [18]:
pd.merge(dt_all_train.drop("Fold", axis = 1), dt_id_folds, on = "ID")

,ID,X0,X1,X10,X100,X101,X102,X103,X104,X105,...,CL_Kmeans_3,CL_DBSCAN_-1,CL_DBSCAN_0,CL_DBSCAN_1,CL_DBSCAN_2,CL_DBSCAN_3,CL_DBSCAN_4,CL_DBSCAN_5,CL_DBSCAN_6,Fold
0,5883,bc,v,0,1,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,6
1,7870,z,w,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,10
2,5158,j,i,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,8
3,1341,z,aa,0,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,2
4,160,az,l,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3
5,1603,z,v,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,8
6,1830,t,aa,0,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,3
7,39,al,r,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1
8,4140,ak,l,0,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,6
9,4019,n,l,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,0,6


In [8]:
dt_all_train.columns.values

array(['ID', 'X0', 'X1', 'X10', 'X100', 'X101', 'X102', 'X103', 'X104',
       'X105', 'X106', 'X107', 'X108', 'X109', 'X11', 'X110', 'X111',
       'X112', 'X113', 'X114', 'X115', 'X116', 'X117', 'X118', 'X12',
       'X120', 'X122', 'X123', 'X124', 'X125', 'X126', 'X127', 'X128',
       'X129', 'X13', 'X131', 'X132', 'X133', 'X135', 'X136', 'X137',
       'X138', 'X139', 'X14', 'X140', 'X141', 'X142', 'X143', 'X144',
       'X145', 'X148', 'X15', 'X150', 'X151', 'X152', 'X153', 'X154',
       'X155', 'X156', 'X158', 'X159', 'X16', 'X160', 'X161', 'X162',
       'X163', 'X164', 'X165', 'X166', 'X167', 'X168', 'X169', 'X17',
       'X170', 'X171', 'X172', 'X173', 'X174', 'X175', 'X176', 'X177',
       'X178', 'X179', 'X18', 'X180', 'X181', 'X182', 'X183', 'X184',
       'X185', 'X186', 'X187', 'X189', 'X19', 'X190', 'X191', 'X192',
       'X194', 'X195', 'X196', 'X197', 'X198', 'X2', 'X20', 'X200', 'X201',
       'X202', 'X203', 'X204', 'X206', 'X207', 'X208', 'X209', 'X21',
       'X2

In [ ]:
dt_withOutlier_aggDupsMedian.shape

In [ ]:
dt_withOutlier_aggDupsMedian.head()

In [ ]:
featureSelectRun("xgb", dt_withOutlier_aggDupsMedian, ids_train, ids_test, cols_cat, cols_bin, rounds = 100, autoFolds = True)